
# Project 5 : Vehicle Detection and Tracking

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import csv
import cv2
import glob
import time
from tqdm import tqdm
import random as rand
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from skimage.feature import hog
%matplotlib inline

### Functions for extracting Features from images 
Below is the function get_hog_features from lesson 34[lesson_functions.py] of Project:Vehicle Detection and Tracking  which will return hog features and visualization.

In [2]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

Below is the function bin_spatial from lesson 22 of Project: Vehicle Detection and Tracking which will return the computed binned color feature

In [3]:
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

Below is the function color_hist from lesson 22 of Project: Vehicle Detection and Tracking which will return the computed color histograms

In [4]:
def color_hist(img, nbins=32, bins_range=(0, 1)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features



Below is the function single_img_features taken from chapter 34 which will  will  extract spatial, color and hog features from single image

In [5]:
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    """Extract spatial, color and hog features from single image
    Args:
        img (numpy.array): image in RGB format
        color_space: GRAY, RGB, HSV, LUV, HLS, YUV, YCrCb
        spatial_size (tuple): resize img before calculating spatial features
            default value is (32, 32)
        hist_bins (int): number of histogram bins, 32 by default
        orient (int): number of HOG orientations
        pix_per_cell (int): number of pixels in HOG cell
        cell_per_block (int): number of HOG cells in block
        hog_channel (int): channel to use for HOG features calculating, default 0
        spatial_feat (boolean): calculate spatial featues, default True
        hist_feat (boolean): calculate histogram featues, default True
        hog_feat (boolean): calculate HOG featues, default True
    Returns:
        features_vector (list(numpy.array)): list of feature vectors
    """
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        feature_image = cv2.cvtColor (img, getattr(cv2, 'COLOR_RGB2' + color_space))
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if color_space == 'GRAY':
            hog_features = get_hog_features(feature_image, orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        elif hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

Below is the function extract_features(taken from chapter 34[lesson_functions.py]) which will  extract spatial, color and hog features from list of images

In [6]:

def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):

    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for image in imgs:
        file_features = []
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
            elif color_space =="GRAY":
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else: 
            feature_image = np.copy(image) 
        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if color_space == 'GRAY':
                hog_features = get_hog_features(feature_image, orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            elif hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

# Loading Vehicle and Non-Vehicle Images
The following code will read images from both vehicle and non-vehicle folder in resources folder

In [7]:
#Loading vehicle and non-vehicle images

cars = glob.glob('resources/vehicles/**/*.png', recursive=True)
notcars = glob.glob('resources/non-vehicles/**/*.png', recursive=True)

images_of_car = [mpimg.imread(impath) for impath in cars]
images_of_noncar = [mpimg.imread(impath) for impath in notcars]

print("{0} images of car found".format(len(images_of_car)))
print("{0} images are non-car images".format(len(images_of_noncar)))

      


8792 images of car found
8968 images are non-car images


## Loading Udacity's  Annotated Driving Dataset

In [8]:
#This if for CrowdAI dataset
# crowdai_car_count=15000
# crowdai_noncar_count = 25000
# crowdai_image_dict={}
# number_of_cars = 0
# with open('resources/crowdai/labels1.csv') as csvfile:
#     reader = csv.reader(csvfile, delimiter=',', quotechar='|')
#     for row in reader:
#         frame = row [4]
#         if frame not in crowdai_image_dict:
#             image = {'impath' : 'resources/crowdai/' + frame,'car_boxes' : []}
#             crowdai_image_dict [frame] = image
#         else:
#             image = crowdai_image_dict [frame]
            
#         if (row[5] == 'Car'):
#             car_data = {
#                 'x1' : int(row [1]),
#                 'y1' : int(row [3]),
#                 'x2' : int(row [0]),
#                 'y2' : int(row [2])
#             }
#             image['car_boxes'].append (car_data)
#             number_of_cars += 1

# print("Number of images in crowdai dataset = {0}".format(len(crowdai_image_dict)))
# print("Number of cars detected = {0}".format(number_of_cars))

In [9]:
autti_car_count=15000
autti_noncar_count = 25000
autti_image_dict={}
number_of_cars = 0

#Make Sure to keep the autti dataset in resources folder
with open('resources/autti/labels.csv', newline ='') as labelcsv:
    reader = csv.reader(labelcsv, delimiter=' ', quotechar='|')
    for row in reader:
        frame = row [0]
        if frame not in autti_image_dict:
            image = {'impath' : 'resources/autti/' + frame,'car_boxes' : []}
            autti_image_dict [frame] = image
        else:
            image = autti_image_dict [frame]
            
        if (row[6] == '"car"'):
            car_data = {
                'x1' : int(row [1]),
                'y1' : int(row [2]),
                'x2' : int(row [3]),
                'y2' : int(row [4])
            }
            image['car_boxes'].append (car_data)
            number_of_cars += 1

print("Number of images in autti dataset = {0}".format(len(autti_image_dict)))
print("Number of cars detected = {0}".format(number_of_cars))

Number of images in autti dataset = 13063
Number of cars detected = 60788


## Croping Images in crowdAI dataset

In [10]:

def non_vehicle_box (image_size, car_boxes):
    non_car_box_size = [128, 64, 48]
    while True:
        box_size = non_car_box_size [rand.randint (0, len (non_car_box_size)-1)]
        box_size = [box_size, box_size]
        temp = [image_size[0] - box_size[0],image_size[1] - box_size[1],]
        position = [rand.randint (0, temp[0] - 1),rand.randint (0, temp[1] - 1)]
        x1, x2, y1, y2 = position [0],position [0] + box_size [0],  position [1], position [1] + box_size [1]
        for each in car_boxes:
            #Checking for intersection 
            boxx1, boxx2, boxy1, boxy2 = each['x1'],each['x2'], each['y1'], each['y2']
            if ((((x1 >= boxx1 and x1 <= boxx2) or (x2 >= boxx1 and x2 <= boxx2)) and
                ((y1 >= boxy1 and y1 <= boxy2) or (y2 >= boxy1 and y2 <= boxy2))) or
            (((boxx1 >= x1 and boxx1 <= x2) or (boxx2 >= x1 and boxx2 <= x2)) and
                ((boxy1 >= y1 and boxy1 <= y2) or (boxy2 >= y1 and boxy2 <= y2)))):
                pass
            else:
                return {'x1':x1,'y1': y1,'x2': x2,'y2': y2}

#For crowdai dataset
# def crop_crowdai_dataset (no_of_cars, no_of_non_cars):
#     images_last_index = len (crowdai_image_dict) - 1
#     cars_crowdai = []
#     non_cars_crowdai = []
#     frames = list(crowdai_image_dict.keys ())
    
#     for i in tqdm(range (max (no_of_cars, no_of_non_cars))):
#         image = crowdai_image_dict [frames[rand.randint (0, len(crowdai_image_dict)-1)]]
#         if len(image ['car_boxes']) == 0:
#             continue
        
        

#         im = mpimg.imread(image ['impath'])
#         im = im.astype(np.float32)/255
#         '''Had to keep the try catch block because opencv has a bug
#             see the below link for the fix
#            https://github.com/opencv/opencv/pull/6883/commits/5bc10ef796c29563e8916ff31b5c1a262422a93f
#         '''
#         try:
        
#             if(i < no_of_cars):
#                 car_box = image['car_boxes'][rand.randint(0, len(image ['car_boxes'])-1)]
#                 car_image = im[car_box['y1']:car_box['y2'], car_box['x1']:car_box['x2']]
#                 car_image = cv2.resize(car_image, (64, 64), interpolation=cv2.INTER_AREA)
#                 cars_crowdai.append (car_image)


#             if(i < no_of_non_cars):
#                 non_car_box = non_vehicle_box ([im.shape[1], im.shape[0]], image['car_boxes'])
#                 non_car_image = im[non_car_box['y1']:non_car_box['y2'], non_car_box['x1']:non_car_box['x2']]
#                 non_car_image = cv2.resize(non_car_image, (64, 64), interpolation=cv2.INTER_AREA)
#                 non_cars_crowdai.append (non_car_image)
#         except:
#             pass
        
#     return(cars_crowdai, non_cars_crowdai)
# cars_crowdai, non_cars_crowdai = crop_crowdai_dataset (crowdai_car_count, crowdai_noncar_count)
# print (' cars:', len (cars_crowdai))
# print ('non cars:', len (non_cars_crowdai))








def crop_autti_dataset (no_of_cars, no_of_non_cars):
    images_last_index = len (autti_image_dict) - 1
    cars_autti = []
    non_cars_autti = []
    frames = list(autti_image_dict.keys ())
    
    for i in tqdm(range (max (no_of_cars, no_of_non_cars))):
        image = autti_image_dict [frames[rand.randint (0, len(autti_image_dict)-1)]]
        if len(image ['car_boxes']) == 0:
            continue
        im = mpimg.imread(image ['impath'])
        im = im.astype(np.float32)/255
        '''Had to keep the try catch block because opencv has a bug
            see the below link for the fix
           https://github.com/opencv/opencv/pull/6883/commits/5bc10ef796c29563e8916ff31b5c1a262422a93f
        '''
        if(i < no_of_cars):
            car_box = image['car_boxes'][rand.randint(0, len(image ['car_boxes'])-1)]
            car_image = im[car_box['y1']:car_box['y2'], car_box['x1']:car_box['x2']]
            car_image = cv2.resize(car_image, (64, 64), interpolation=cv2.INTER_AREA)
            cars_autti.append (car_image)


        if(i < no_of_non_cars):
            non_car_box = non_vehicle_box ([im.shape[1], im.shape[0]], image['car_boxes'])
            non_car_image = im[non_car_box['y1']:non_car_box['y2'], non_car_box['x1']:non_car_box['x2']]
            non_car_image = cv2.resize(non_car_image, (64, 64), interpolation=cv2.INTER_AREA)
            non_cars_autti.append (non_car_image)

    return(cars_autti, non_cars_autti)

cars_autti, non_cars_autti = crop_autti_dataset (autti_car_count, autti_noncar_count)
print (' cars:', len (cars_autti))
print ('non cars:', len (non_cars_autti))



100%|███████████████████████████████████████████████████████████████████████| 500/500 [01:01<00:00,  7.72it/s]


 cars: 492
non cars: 492


## Extracting Features, Normalizing Features, Training the Classifier

In [11]:

# parameters of feature extraction

color_space = 'GRAY' # Can be GRAY, RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 8  # HOG orientations
pix_per_cell = 16 # HOG pixels per cell
cell_per_block = 1 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = False # Spatial features on or off
hist_feat = False # Histogram features on or off
hog_feat = True # HOG features on or off

#For crowdai dataset
# # Combining crowdai, vehicle and non-vehicle resources
# all_cars = []
# all_cars.extend(cars)
# all_cars.extend(cars_crowdai)

# all_non_cars = []
# all_non_cars.extend(notcars)
# all_non_cars.extend(non_cars_crowdai)


#for autti dataset

all_cars = []
all_cars.extend(images_of_car)
all_cars.extend(cars_autti)

all_non_cars = []
all_non_cars.extend(images_of_noncar)
all_non_cars.extend(non_cars_autti)



#Extracting Features
ft=time.time()
car_features = extract_features(all_cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(all_non_cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
ft2=time.time()
print ('features extraction time: ', round(ft2-ft, 2))


#Normalizing features
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        

X_scaler = StandardScaler().fit(X)

scaled_X = X_scaler.transform(X)


y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

#Training the classifier
#Test Accuracy of LinearSVC =  0.9384  
#Test Accuracy of LinearSVC(loss = 'hinge') =  0.9419
#SVC is showing better Accuracy, so taking SVC
svc = SVC()

# Timing the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))


C:\Program Files\Anaconda3\envs\py35\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


features extraction time:  21.25
Using: 8 orientations 16 pixels per cell and 1 cells per block
Feature vector length: 128
Test Accuracy of SVC =  0.9829


## Sliding Window

The below function slide_window is taken from Chapter 32 of Vehicle Detection and Tracking Lesson.
This function takes an image,start and stop positions in both x and y,window size (x and y dimensions) and overlap fraction (for both x and y)

In [12]:
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

## Search Window

In [13]:
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):
    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64), interpolation=cv2.INTER_AREA)      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier

        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

## Function to draw Boxes

In [14]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):

    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

def get_s_from_hls (img):
    hls = cv2.cvtColor (img, cv2.COLOR_BGR2HLS)
    return hls [:,:,2]

In [15]:
image = mpimg.imread('test_images/test1.jpg')
window_img = np.copy(image)

sw_x_limits = [
    [None, None],
    [32, None],
    [412, 1280]
]

sw_y_limits = [
    [400, 640],
    [400, 600],
    [390, 540]
]

sw_window_size = [
    (128, 128),
    (96, 96),
    (80, 80)
]

sw_overlap = [
    (0.5, 0.5),
    (0.5, 0.5),
    (0.5, 0.5)
]

# create sliding windows
windows = slide_window(image, x_start_stop=sw_x_limits[0], y_start_stop=sw_y_limits[0], 
                    xy_window=sw_window_size[0], xy_overlap=sw_overlap[0])

windows2 = slide_window(image, x_start_stop=sw_x_limits[1], y_start_stop=sw_y_limits[1], 
                    xy_window=sw_window_size[1], xy_overlap=sw_overlap[1])

windows3 = slide_window(image, x_start_stop=sw_x_limits[2], y_start_stop=sw_y_limits[2], 
                    xy_window=sw_window_size[2], xy_overlap=sw_overlap[2])

# show sliding windows

sliding_windows= draw_boxes(np.copy(image), windows, color=(0, 0, 0), thick=4)


# drawing one of sliding windows in blue
sliding_windows = draw_boxes (sliding_windows, [windows[9]], color=(0, 0, 255), thick=8)



In [16]:
def get_hot_boxes (image):


    dst = np.copy (image)
    all_hot_windows = []
    
    # iterate over previousely defined sliding windows
    for x_limits, y_limits, window_size, overlap in zip (sw_x_limits, sw_y_limits, sw_window_size, sw_overlap):

        windows = slide_window(
            dst,
            x_start_stop=x_limits,
            y_start_stop=y_limits, 
            xy_window=window_size,
            xy_overlap=overlap
        )

        hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       
        
        all_hot_windows.extend (hot_windows)

        dst = draw_boxes(dst, hot_windows, color=(0, 0, 1), thick=4)

    return all_hot_windows, dst
        
def get_heat_map(image, bbox_list):


    heatmap = np.zeros_like(image[:,:,0]).astype(np.float)

    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    
    # Return updated heatmap
    return heatmap

class HotBox ():

    def __init__ (self, box):
        self.avg_box = [list(p) for p in box]
        self.detected_count = 1
        self.boxes = [box]
    
    def get_strength (self):

        return self.detected_count
    
    def get_box (self):

        if len(self.boxes) > 1:
            center = np.average (np.average (self.boxes, axis=1), axis=0).astype(np.int32).tolist()

            # getting all x and y coordinates of
            # all corners of joined boxes separately
            xs = np.array(self.boxes) [:,:,0]
            ys = np.array(self.boxes) [:,:,1]

            half_width = int(np.std (xs))
            half_height = int(np.std (ys))
            return (
                (
                    center[0] - half_width,
                    center[1] - half_height
                ), (
                    center[0] + half_width,
                    center[1] + half_height
                ))
        else:
            return self.boxes [0]
    
    def is_close (self, box):
        x11 = self.avg_box [0][0]
        y11 = self.avg_box [0][1]
        x12 = self.avg_box [1][0]
        y12 = self.avg_box [1][1]
        x21 = box [0][0]
        y21 = box [0][1]
        x22 = box [1][0]
        y22 = box [1][1]
            
        x_overlap = max(0, min(x12,x22) - max(x11,x21))
        y_overlap = max(0, min(y12,y22) - max(y11,y21))

        area1 = (x12 - x11) * (y12 - y11)
        area2 = (x22 - x21) * (y22 - y21)
        intersection = x_overlap * y_overlap;
        
        if (
            intersection >= 0.3 * area1 or
            intersection >= 0.3 * area2
        ):
            return True
        else:
            return False
    
    def join (self, boxes):
        
        joined = False
        
        for b in boxes:
            if self.is_close (b):
                boxes.remove (b)
                self.boxes.append (b)
                self.detected_count += 1
                
                self.avg_box [0][0] = min (self.avg_box [0][0], b [0][0])
                self.avg_box [0][1] = min (self.avg_box [0][1], b [0][1])
                self.avg_box [1][0] = max (self.avg_box [1][0], b [1][0])
                self.avg_box [1][1] = max (self.avg_box [1][1], b [1][1])
                
                joined = True

        return joined

def calc_average_boxes (hot_boxes, strength):

    avg_boxes = []
    while len(hot_boxes) > 0:
        b = hot_boxes.pop (0)
        hb = HotBox (b)
        while hb.join (hot_boxes):
            pass
        avg_boxes.append (hb)
    
    boxes = []
    for ab in avg_boxes:
        if ab.get_strength () >= strength:
            boxes.append (ab.get_box ())
    return boxes

In [17]:
test_images = []
test_images_titles = []

for impath in glob.glob('test_images/test*.jpg'):
    image_orig = mpimg.imread(impath)
    

    image = image_orig.astype(np.float32)/255

    # hot boxes
    hot_boxes, image_with_hot_boxes = get_hot_boxes (image)
    # heat map
    heat_map = get_heat_map (image, hot_boxes)
    
    # average boxes
    avg_boxes = calc_average_boxes (hot_boxes, 2)
    image_with_boxes = draw_boxes(image, avg_boxes, color=(0, 0, 1), thick=4)
    
    test_images.append (image_with_hot_boxes)
    test_images.append (heat_map)
    test_images.append (image_with_boxes)
    
    test_images_titles.extend (['', '', ''])
    
test_images_titles [0] = 'hot boxes'
test_images_titles [1] = 'heat map'
test_images_titles [2] = 'average boxes'


C:\Program Files\Anaconda3\envs\py35\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [18]:
class LastHotBoxes:

    def __init__ (self,max_len):
        self.queue_max_len = max_len
        self.last_boxes = []

    def put_hot_boxes (self, boxes):

        if (len(self.last_boxes) > self.queue_max_len):
            tmp = self.last_boxes.pop (0)
        
        self.last_boxes.append (boxes)
        
    def get_hot_boxes (self):

        b = []
        for boxes in self.last_boxes:
            b.extend (boxes)
        return b

last_hot_boxes = LastHotBoxes (15)
    
def process_image (image_orig):
    
    image_orig = np.copy (image_orig)
    image = image_orig.astype(np.float32)/255

    hot_boxes, image_with_hot_boxes = get_hot_boxes (image)
    last_hot_boxes.put_hot_boxes (hot_boxes)
    hot_boxes = last_hot_boxes.get_hot_boxes ()
    

    avg_boxes = calc_average_boxes (hot_boxes, 20)
    image_with_boxes = draw_boxes(image, avg_boxes, color=(0, 0, 1), thick=4)

    return image_with_boxes * 255

In [19]:
from moviepy.editor import VideoFileClip

def process_video (input_path, output_path):
    clip = VideoFileClip (input_path)
    


    result = clip.fl_image (process_image)
    %time result.write_videofile (output_path)

# select video to operate on
# process_video ('test_video.mp4', 'output/test_video_result.mp4')
process_video ('project_video.mp4', 'output/project_video_result.mp4')
# process_video ('challenge_video.mp4', 'output/challenge_video_result.mp4')
# process_video ('harder_challenge_video.mp4', 'output/harder_challenge_video_result.mp4')


[MoviePy] >>>> Building video output/project_video_result.mp4
[MoviePy] Writing audio in project_video_resultTEMP_MPY_wvf_snd.mp3


100%|████████████████████████████████████████████████████████████████████| 1112/1112 [00:01<00:00, 979.46it/s]


[MoviePy] Done.
[MoviePy] Writing video output/project_video_result.mp4


100%|████████████████████████████████████████████████████████████████████▉| 1260/1261 [08:11<00:00,  2.79it/s]


[MoviePy] Done.


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'project_video_resultTEMP_MPY_wvf_snd.mp3'